# Where we left off
In Part 02 we registered for an API key, stored it in our environment variables, and added logic to our `StackOverflow.__init__` method to add it to our API calls increasing our `quota_max` and `quota_remaining`.

# Tracking your remaining quota
How do I know my `quota_remaining` without making an API call? You don't. But we could store the value in `StackOverflow` object so we don't have to make more than one API call.

To do this we'll need to add a `quota_remaining` attribute and modify the current `fetch` method:

In [1]:
"""contents of api.py"""
from os import getenv

from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

    def __init__(self, key: str | None = None, **kwargs):
        if key is None:
            key = getenv(key="stackapi_key")
        super().__init__(name="stackoverflow", key=key, **kwargs)
        self.quota_remaining: int | None = None

    def fetch(self, endpoint=None, page=1, key=None, filter='default', **kwargs):
        response = super().fetch(
            endpoint=endpoint, page=page, key=key, filter=filter, **kwargs
        )
        self.quota_remaining = response.get("quota_remaining")
        return response

In [2]:
so = StackOverflow()

response = so.fetch("questions")
print(so.quota_remaining)

9940


It works! But I'm not satisfied. Because we added the attribute `quota_remaining` to the class in the way we did, we could technically overwrite it and be none the wiser.

In [3]:
so.quota_remaining = 10_000
print(so.quota_remaining)

10000


How do we prevent this? By using a `@property` and making the value private.

In [4]:
"""contents of api.py"""
from os import getenv

from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

    def __init__(self, key: str | None = None, **kwargs):
        if key is None:
            key = getenv(key="stackapi_key")
        super().__init__(name="stackoverflow", key=key, **kwargs)
        self._quota_remaining: int | None = None

    def fetch(self, endpoint=None, page=1, key=None, filter='default', **kwargs):
        response = super().fetch(
            endpoint=endpoint, page=page, key=key, filter=filter, **kwargs
        )
        self._quota_remaining = response.get("quota_remaining")
        return response

    @property
    def quota_remaining(self) -> int | None:
        return self._quota_remaining

Running the code before we'll get our actual `quota_remaining`:

In [5]:
so = StackOverflow()

response = so.fetch("questions")
print(so.quota_remaining)

9935


But this time when we try to overwrite it we'll raise an error:

In [6]:
so.quota_remaining = 10_000

AttributeError: property 'quota_remaining' of 'StackOverflow' object has no setter

Much better. Now what about our `key`? Does the same thing happen?

In [9]:
so.key = "123456"
print(so.key)

123456


It does! If we accidentally overwrote our `key` mid-script would we still be able to make API calls?

In [10]:
so.fetch("questions")

StackAPIError: ('https://api.stackexchange.com/2.2/questions/?pagesize=100&page=1&filter=default&key=123456&site=stackoverflow', 400, 'bad_parameter', "`key` doesn't match a known application")

Unfortunately not. However, if we make the `key` private and add a property to make it un-assignable we lose the flexibility of changing to a different `key` without defining a new `StackOverflow` object. With this in mind, it is probably better to _not_ privatize the key.